This notebook solve the problem presented in the kaggle competition : https://www.kaggle.com/competitions/store-sales-time-series-forecasting/overview

The objective of this project is to **forecast future product sales** for a large
number of time series, where each series corresponds to a unique combination of
**store** and **product family**.

Given historical daily sales data and known future information
(promotions, calendar effects, oil prices, holidays, and store metadata),
the goal is to **predict sales for the next 16 days** for every
(store, family) pair.

Metric used : $
\mathrm{RMSLE}(y, \hat{y}) =
\sqrt{
\frac{1}{n}
\sum_{i=1}^{n}
\left(
\log(1 + \hat{y}_i) - \log(1 + y_i)
\right)^2
}
$


In [44]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression,ElasticNet,Ridge,Lasso
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier

from pathlib import Path
import sys

ROOT = Path.cwd()
if not (ROOT / "src").exists() and (ROOT.parent / "src").exists():
    ROOT = ROOT.parent
sys.path.insert(0, str(ROOT))

from src.io import load_train, load_test, load_oil, load_holidays, load_stores
from src.preprocessing import add_calendar_features, join_oil, build_holiday_features, join_holidays, join_stores
from src.utils import make_group_lags, make_group_leads, make_group_rolling
from src.models import HybridModel
from src.validation import rmsle, time_train_valid_split
from src.viz import *

## 1. Data Preparation

In [45]:
# --- Load data ---
train = load_train()
test = load_test()

oil = load_oil()
holidays = load_holidays()
stores = load_stores()
stores["store_nbr"] = stores["store_nbr"].astype(str)

HORIZON = 16

print("Train shape:", train.shape)
print("Test shape :", test.shape)

Train shape: (3000888, 2)
Test shape : (28512, 1)


In [46]:
y_wide = (
    train["sales"]
    .unstack(["store_nbr", "family"])
    .sort_index()
)

print("y_wide shape:", y_wide.shape)
print("n series (columns):", y_wide.shape[1])
print("column levels:", y_wide.columns.nlevels, y_wide.columns.names)

y_wide shape: (1684, 1782)
n series (columns): 1782
column levels: 2 ['store_nbr', 'family']


In [47]:
# ------------------------------------------------------------
# DeterministicProcess: trend + Fourier seasonality (global)
# ------------------------------------------------------------

fourier_week = CalendarFourier(freq="W", order=3)

dp = DeterministicProcess(
    index=y_wide.index,
    constant=True,
    order=1,  
    additional_terms=[fourier_week],
    drop=True,
)

X1_train = dp.in_sample()

X1_test = dp.out_of_sample(steps=HORIZON)
X1_test.index.name = "date"



In [52]:
# ------------------------------------------------------------
# Base data : oil, holidays dummies, calendar features.
# ------------------------------------------------------------

X2_train = train[["onpromotion"]].copy()
X2_train = X2_train.reorder_levels(["date", "store_nbr", "family"]).sort_index()

X2_test = test[["onpromotion"]].copy()
X2_test = X2_test.reorder_levels(["date", "store_nbr", "family"]).sort_index()



# Calendar features (ONLY for Model 2)
X2_train = add_calendar_features(X2_train, date_level="date")
X2_test = add_calendar_features(X2_test, date_level="date")

# Oil
X2_train = join_oil(X2_train, oil, date_level="date")
X2_test = join_oil(X2_test, oil, date_level="date")


# Holidays
dates_tr = X2_train.index.get_level_values("date")
holiday_feats_tr = build_holiday_features(holidays, start=dates_tr.min(), end=dates_tr.max())
X2_train = join_holidays(X2_train, holiday_feats_tr, date_level="date")

dates_te = X2_test.index.get_level_values("date")
holiday_feats_te = build_holiday_features(holidays, start=dates_te.min(), end=dates_te.max())
X2_test = join_holidays(X2_test, holiday_feats_te, date_level="date")


# Stores
X2_train = join_stores(X2_train, stores)
X2_test = join_stores(X2_test, stores)


X2_train.index = X2_train.index.set_names(["date", "store_nbr", "family"])
X2_test.index = X2_test.index.set_names(["date", "store_nbr", "family"])

# ------------------------------------------------------------
# Add dynamic features:  rolling stats + promo dynamics
# ------------------------------------------------------------

# Prepare log-sales series in long format with index (date, store_nbr, family)
y_log_long = np.log1p(train["sales"]).copy()
y_log_long = y_log_long.reorder_levels(["date", "store_nbr", "family"]).sort_index()
y_log_long.name = "ylog"


# Use ONLY training period to compute means (no leakage)
enc_train = y_log_long.loc[y_wide.index]
global_mean = enc_train.mean()

# Store average
store_avg = enc_train.groupby(level="store_nbr").mean()

# Family average
family_avg = enc_train.groupby(level="family").mean()


# Apply to train
store_idx_tr = X2_train.index.get_level_values("store_nbr")
family_idx_tr = X2_train.index.get_level_values("family")

alpha = 20

store_stats = enc_train.groupby(level="store_nbr").agg(["mean", "count"])
family_stats = enc_train.groupby(level="family").agg(["mean", "count"])

store_smooth = (
    store_stats["mean"] * store_stats["count"] + global_mean * alpha
) / (store_stats["count"] + alpha)

family_smooth = (
    family_stats["mean"] * family_stats["count"] + global_mean * alpha
) / (family_stats["count"] + alpha)

X2_train["store_avg"]  = store_idx_tr.map(store_smooth).astype("float32").fillna(global_mean)
X2_train["family_avg"] = family_idx_tr.map(family_smooth).astype("float32").fillna(global_mean)

# Apply to test
store_idx_te = X2_test.index.get_level_values("store_nbr")
family_idx_te = X2_test.index.get_level_values("family")

X2_test["store_avg"]  = store_idx_te.map(store_smooth).astype("float32").fillna(global_mean)
X2_test["family_avg"] = family_idx_te.map(family_smooth).astype("float32").fillna(global_mean)




In [53]:
print(X1_test.index.min(), X1_test.index.max(), len(X1_test))
print(X2_test.index.get_level_values("date").min(),
      X2_test.index.get_level_values("date").max(),
      X2_test.index.get_level_values("date").nunique())


2017-08-16 2017-08-31 16
2017-08-16 2017-08-31 16


## 2. Logic and Structure

$$
y_{s,f,t} = T_t + S_t + C_{s,f,t} + \varepsilon_{s,f,t}
$$

- **$T_t$**: global trend (slow-moving component)  
- **$S_t$**: global seasonality (weekly/yearly) + holiday effects  
- **$C_{s,f,t}$**: local cycle / autocorrelation (lags, rolling stats) and nonlinear interactions with promotions  
- **$\varepsilon_{s,f,t}$**: remaining noise  



### Hybrid model

1. **Model 1 (linear):** learns deterministic structure  
   $$
   T_t + S_t
   $$
   from time features (*DeterministicProcess + Fourier terms*).

2. **Model 2 (tree model):** learns residual structure using tabular features  
   (lags, promotions, oil, calendar indicators).



We train in **log space** using  
$$
\log(1 + y)
$$
to align with the **RMSLE** metric.


In [55]:
y_global = (
    train["sales"]
    .groupby("date")
    .mean()
    .sort_index()
)

y_global.name = "average_sales"

y_global.head(), y_global.index.min(), y_global.index.max()

(date
 2013-01-01      1.409438
 2013-01-02    278.390808
 2013-01-03    202.840195
 2013-01-04    198.911163
 2013-01-05    267.873260
 Freq: D, Name: average_sales, dtype: float32,
 Period('2013-01-01', 'D'),
 Period('2017-08-15', 'D'))

In [ ]:
trend = y_global.rolling(
    window=365,
    center=True,
    min_periods=183,
).mean()

ax = y_global.plot(**plot_params, alpha=0.5, title="Global average sales (daily)", ylabel= "avg sales")
ax = trend.plot(ax=ax, linewidth=3)

In [ ]:
y1 = np.log1p(y_global)  # Linear model: DP owns intercept => fit_intercept=False
det_model = LinearRegression(fit_intercept=False)
det_model.fit(X1, y1)

X1_fore = dp.out_of_sample(steps=HORIZON) 

y1_fit = pd.Series(det_model.predict(X1), index=X1.index, name="fit_log")
y1_fore = pd.Series(det_model.predict(X1_fore), index=X1_fore.index, name="fore_log")

# Convert back to sales scale
y_fit = np.expm1(y1_fit).rename("fit")
y_fore = np.expm1(y1_fore).rename("forecast")

global_residual = (y1 - y1_fit).rename("resid_log")

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(10, 7))
ax1 = plot_periodogram(y1, ax=ax1)
ax1.set_title("Product Sales Frequency Components")
ax2 = plot_periodogram(global_residual, ax=ax2)
ax2.set_title("Deseasonalized")


Fit the hybrid model

In [58]:
model1 = LinearRegression(fit_intercept=False)
model2 = LGBMRegressor(
    n_estimators=1000,          
    learning_rate=0.05,        
    num_leaves=31,             
    max_depth=8,               
    min_child_samples=200,     
    subsample=0.7,
    colsample_bytree=0.7,
    reg_alpha=1.0,             
    reg_lambda=5.0,           
    random_state=42,
    n_jobs=-1,
)

hybrid = HybridModel(model1=model1, model2=model2, transform="log1p", clip=True)

hybrid.fit(X1=X1_train,X2=X2_train,y_wide=y_wide)

y_pred_train = hybrid.predict(X1_train, X2_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.035888 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 722
[LightGBM] [Info] Number of data points in the train set: 3000888, number of used features: 15
[LightGBM] [Info] Start training from score 0.000000


In [59]:
print("Train RMSLE:", rmsle(y_wide.loc[y_pred_train.index], y_pred_train))

Train RMSLE: 0.5473666451981261


##  3. SUBMISSION

In [ ]:
y_pred_test = hybrid.predict(X1_test, X2_test)
y_pred_test.shape

(16, 1782)

In [ ]:
y_pred_test

store_nbr           1                                                          \
family     AUTOMOTIVE BABY CARE    BEAUTY    BEVERAGES     BOOKS BREAD/BAKERY   
date                                                                            
2017-08-16   4.077220  0.000000  3.552902  2837.947804  0.237753   472.198427   
2017-08-17   3.624448  0.000000  3.433444  2423.785978  0.206639   406.257027   
2017-08-18   3.733552  0.000000  2.762890  2434.243841  0.184922   378.033017   
2017-08-19   3.983760  0.000000  3.355442  2662.808534  0.190093   373.133794   
2017-08-20   1.658525  0.000000  1.884443  1155.201801  0.208660   168.015472   
2017-08-21   3.395438  0.000000  3.070920  2501.960453  0.175944   405.097204   
2017-08-22   3.789142  0.000000  2.460367  2256.667473  0.156343   353.642887   
2017-08-23   3.416302  0.000000  2.981306  2599.049387  0.177604   433.754610   
2017-08-24   3.641389  0.000000  2.868099  2415.500684  0.198465   408.727592   
2017-08-25   3.792984  0.000000  2.269040  2454.787358  0.173252   387.212314   
2017-08-26   4.616863  0.000000  3.772841  2805.836820  0.235510   398.479548   
2017-08-27   1.899382  0.003676  2.035817  1182.179778  0.226743   172.508308   
2017-08-28   3.640410  0.000000  3.183726  2504.671085  0.157923   405.415235   
2017-08-29   4.239057  0.000000  3.245978  2347.055929  0.179011   367.176045   
2017-08-30   4.022434  0.000000  2.837339  2886.027441  0.225681   476.071745   
2017-08-31   3.683690  0.000000  3.481559  2497.628962  0.209161   418.824780   

store_nbr                                                   ...         9  \
family     CELEBRATION    CLEANING       DAIRY        DELI  ... MAGAZINES   
date                                                        ...             
2017-08-16   29.843061  828.679873  994.707862  150.338503  ...  3.576384   
2017-08-17   31.132732  668.690190  766.096137  123.232758  ...  3.608864   
2017-08-18   26.487539  648.722017  768.846247  156.572862  ...  3.327993   
2017-08-19   20.908269  558.474028  827.898548  130.084569  ...  4.135750   
2017-08-20   10.178960  244.959053  379.139041   58.511633  ...  4.438169   
2017-08-21   20.580962  634.416772  803.759040  137.160302  ...  3.093913   
2017-08-22   21.494621  608.875006  729.125321  120.793933  ...  2.999320   
2017-08-23   23.475445  760.349980  902.853404  134.848461  ...  3.016657   
2017-08-24   31.722852  669.061461  766.651351  122.942299  ...  3.642916   
2017-08-25   27.214201  656.813953  778.566225  157.459695  ...  3.338948   
2017-08-26   26.811886  589.668330  876.033638  137.600518  ...  4.795949   
2017-08-27   12.732565  241.166504  390.489691   59.987452  ...  5.090425   
2017-08-28   24.723117  636.350518  807.977886  138.009120  ...  3.339449   
2017-08-29   27.384901  634.534151  761.511827  126.002808  ...  3.391212   
2017-08-30   32.238712  835.130987  993.807596  147.250380  ...  3.594726   
2017-08-31   32.347532  688.908864  791.122264  126.260796  ...  3.663196   

store_nbr                                                                  \
family           MEATS PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS   
date                                                                        
2017-08-16  310.262146    421.718669     8.496319               16.952038   
2017-08-17  468.674056    401.351246     7.895784               17.635558   
2017-08-18  247.059638    323.072879     6.774765               12.647951   
2017-08-19  361.264323    581.838451     9.567968               18.219171   
2017-08-20  404.782907    663.492519    10.755160               20.420218   
2017-08-21  340.241307    464.927665     7.430536               15.348998   
2017-08-22  293.302095    407.103239     7.199527               13.780498   
2017-08-23  274.178546    361.519930     7.353691               13.158501   
2017-08-24  460.240621    396.046240     7.981774               17.823251   
2017-08-25  236.952094    318.900491     6.812069               12.85864

In [ ]:
pred_flat = (
    y_pred_test
    .stack(["store_nbr", "family"])
    .values
)

(28512,)

In [86]:
submission = pd.read_csv(ROOT/"Data/sample_submission.csv")
submission["sales"] = pred_flat
submission.to_csv("submission.csv", index=False)

print("submission.csv saved successfully!")

submission.csv saved successfully!


In [87]:
print(submission.shape)          # (28512, 2)
print(submission.head())
print(submission.isna().sum())   # 0

(28512, 2)
        id        sales
0  3000888     4.077220
1  3000889     0.000000
2  3000890     3.552902
3  3000891  2837.947804
4  3000892     0.237753
id       0
sales    0
dtype: int64
